<a href="https://colab.research.google.com/github/SamurAIGPT/AI-Influencer/blob/main/AI_Influencer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Autor: André Leli 
 - Ultima atualização: Julho 2025

# 🎥 Gerador de Pessoa Virtual com IA

Este notebook usa:
- **Stable Diffusion** → para gerar o rosto do influenciador
- **gTTS (Google Text-to-Speech)** → para gerar a narração a partir do texto
- **SadTalker** → para animar o rosto com sincronização labial com a fala

___________

## ✅ Etapas de Configuração  
**(Rodar no terminal do Windows / Bash / Linux / Google Colab)**

> Execute os comandos abaixo **fora do notebook**, no terminal do sistema ou na primeira célula do Colab.

### 🔹 1. Instale o Python 3.8 ou 3.9 manualmente  
- Acesse: [https://www.python.org/downloads/](https://www.python.org/downloads/)  
- Baixe e instale a versão 3.8 ou 3.9  
- Durante a instalação, marque a opção: **"Add Python to PATH"**

.......

### 🔹 2. Verifique a instalação do Python  
```bash
    python --version
```

.......


### 🔹 3. Atualize o `pip`  
```bash
    python -m pip install --upgrade pip --user
    pip --version
```

.......

### 🔹 4. (Opcional, mas recomendado) Crie e ative um ambiente virtual  
```bash
    python -m venv venv
    venv\Scripts\activate
```

.......


### 🔹 5. Instale o Git (caso ainda não tenha)  
- Acesse: [https://git-scm.com/download/win](https://git-scm.com/download/win)  
- Faça a instalação padrão

.......


### 🔹 6. Clone o repositório SadTalker  
```bash
    git clone https://github.com/Winfredy/SadTalker
    cd SadTalker
```

.......


### 🔹 7. Instale o FFmpeg (manual)  
- Acesse: [https://www.gyan.dev/ffmpeg/builds/](https://www.gyan.dev/ffmpeg/builds/)  
- Baixe e extraia o `.zip`  
- Copie o caminho da pasta `bin` e adicione à variável de ambiente **PATH**  
- Reinicie o terminal e teste:
```bash
    ffmpeg -version
```

.......


### 🔹 8. Instale o PyTorch compatível com sua GPU (ou CPU)

#### Exemplo para CUDA 11.3:
```bash
    pip install torch==2.2.2+cu113 torchvision==0.17.2+cu113 torchaudio==2.2.2 --index-url https://download.pytorch.org/whl/cu113 --user
```

#### CPU (sem GPU):
```bash
    pip install torch==2.2.2 torchvision==0.17.2 torchaudio==2.2.2 --index-url https://download.pytorch.org/whl/cpu --user
```

#### Instale dependências adicionais:
```bash
    pip install basicsr==1.4.2 gfpgan==1.3.8 --user
```

#### Se tiver problemas, tente CUDA 12.1:
```bash
    pip uninstall torch torchvision torchaudio -y
    pip cache purge
    pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
```

#### Ou alternativa com CUDA 11.8:
```bash
    pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
```

.......


### 🔹 9. Instale os requisitos do SadTalker  
```bash
    pip install -r requirements.txt --user
```

.......


### 🔹 10. Instale bibliotecas auxiliares  
```bash
    pip install openai gtts diffusers accelerate transformers requests safetensors --user
```

.......


### 🔹 11. API do Modelo - OpenAI 

- Não se esqueça de gerar uma chave de API da Open AI e colocar no arquivo ".env" como valor da chave "API_KEY"


________

### Baixando modelos de processamento:

In [17]:
import os
import requests

BASE_DIR = os.path.join(os.getcwd(), "SadTalker")

def baixar_arquivo(url, destino):
    if os.path.exists(destino):
        print(f'[SKIP] Já existe: {destino}')
        return
    os.makedirs(os.path.dirname(destino), exist_ok=True)
    print(f'[DOWNLOADING] {url} → {destino}')
    try:
        with requests.get(url, stream=True, timeout=120) as r:
            r.raise_for_status()
            with open(destino, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
        print(f'[OK] Download concluído: {destino}')
    except Exception as e:
        print(f'[ERRO] Falha ao baixar {url}\n{e}')

# Modelos principais para SadTalker/checkpoints
modelos_checkpoints = {
    "mapping_00109-model.pth.tar": "https://github.com/OpenTalker/SadTalker/releases/download/v0.0.2-rc/mapping_00109-model.pth.tar",
    "mapping_00229-model.pth.tar": "https://github.com/OpenTalker/SadTalker/releases/download/v0.0.2-rc/mapping_00229-model.pth.tar",
    "SadTalker_V0.0.2_256.safetensors": "https://github.com/OpenTalker/SadTalker/releases/download/v0.0.2-rc/SadTalker_V0.0.2_256.safetensors",
    "SadTalker_V0.0.2_512.safetensors": "https://github.com/OpenTalker/SadTalker/releases/download/v0.0.2-rc/SadTalker_V0.0.2_512.safetensors",
    "epoch_20.pth": "https://huggingface.co/camenduru/SadTalker/resolve/main/epoch_20.pth",
}

# Modelo BFM para SadTalker/checkpoints/BFM_Fitting
bfm_dir = os.path.join(BASE_DIR, "checkpoints", "BFM_Fitting")
os.makedirs(bfm_dir, exist_ok=True)
bfm_path = os.path.join(bfm_dir, "BFM_model_front.mat")
bfm_url = "https://huggingface.co/camenduru/video-retalking/resolve/main/BFM/BFM_model_front.mat"

# Modelos para SadTalker/gfpgan/weights
modelos_gfpgan = {
    "alignment_WFLW_4HG.pth": "https://github.com/xinntao/facexlib/releases/download/v0.1.0/alignment_WFLW_4HG.pth",
    "detection_Resnet50_Final.pth": "https://github.com/xinntao/facexlib/releases/download/v0.1.0/detection_Resnet50_Final.pth",
    "GFPGANv1.4.pth": "https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.4.pth",
    "parsing_parsenet.pth": "https://github.com/xinntao/facexlib/releases/download/v0.2.2/parsing_parsenet.pth",
}

# Baixar modelos principais (SadTalker/checkpoints)
for nome, url in modelos_checkpoints.items():
    destino = os.path.join(BASE_DIR, "checkpoints", nome)
    baixar_arquivo(url, destino)

# Baixar modelo BFM_model_front.mat (SadTalker/checkpoints/BFM_Fitting)
baixar_arquivo(bfm_url, bfm_path)

# Baixar modelos para gfpgan/weights
for nome, url in modelos_gfpgan.items():
    destino = os.path.join(BASE_DIR, "gfpgan", "weights", nome)
    baixar_arquivo(url, destino)


[SKIP] Já existe: c:\Users\andre\OneDrive\Área de Trabalho\ESTUDOS\IA_GEN\AI-Influencer-Generator\SadTalker\checkpoints\mapping_00109-model.pth.tar
[SKIP] Já existe: c:\Users\andre\OneDrive\Área de Trabalho\ESTUDOS\IA_GEN\AI-Influencer-Generator\SadTalker\checkpoints\mapping_00229-model.pth.tar
[SKIP] Já existe: c:\Users\andre\OneDrive\Área de Trabalho\ESTUDOS\IA_GEN\AI-Influencer-Generator\SadTalker\checkpoints\SadTalker_V0.0.2_256.safetensors
[SKIP] Já existe: c:\Users\andre\OneDrive\Área de Trabalho\ESTUDOS\IA_GEN\AI-Influencer-Generator\SadTalker\checkpoints\SadTalker_V0.0.2_512.safetensors
[SKIP] Já existe: c:\Users\andre\OneDrive\Área de Trabalho\ESTUDOS\IA_GEN\AI-Influencer-Generator\SadTalker\checkpoints\epoch_20.pth
[DOWNLOADING] https://huggingface.co/camenduru/video-retalking/resolve/main/BFM/BFM_model_front.mat → c:\Users\andre\OneDrive\Área de Trabalho\ESTUDOS\IA_GEN\AI-Influencer-Generator\SadTalker\checkpoints\BFM_Fitting\BFM_model_front.mat
[OK] Download concluído: c:\U

### Conectando no GPT para geração de prompt:

In [ ]:
import requests
import json
from openai import OpenAI

import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())


client = OpenAI(
    api_key=os.environ.get("API_KEY")
)

def get_prompt_for_image(caracteristicas: str) -> str:
    """
    Gera um prompt em inglês, estratégico e compacto, para IA de geração de imagens realistas,
    priorizando branding, autoridade e uso corporativo.

    Parâmetros:
        caracteristicas (str): Descrição detalhada do avatar (pode estar em PT ou EN).

    Retorno:
        str: Prompt refinado, 100% em inglês, para modelos de imagem.
    """
    system_prompt = (
        "You are a world-class prompt engineer for AI image generation. "
        "Your job is to convert high-level avatar descriptions into powerful, concise, and hyper-realistic prompts in natural English, "
        "always optimized for corporate branding, business authority and flawless realism. "
        "IMPORTANT: Never exceed 75 words and keep the structure tight for the best CLIP/Stable Diffusion results. "
        "Do NOT output any markdown or commentary, only a valid JSON as shown."
    )
    user_prompt = f"""
Characteristics of the avatar (can be in PT or EN):
{caracteristicas}

Guidelines:
- Hyper-realistic, professional studio portrait photo, front-facing.
- Show face, shoulders, bust; direct gaze and authentic smile.
- Confident, empathetic expression.
- Soft, elegant studio lighting, neutral/professional background.
- Sophisticated, modern, business attire.
- Absolutely NO cartoon, distortion, clutter, or ambiguity.

Format (respond ONLY this JSON, nothing else):
{{
  "prompt": "<your ultra-concise, natural English prompt here>"
}}
    """

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.25,
        max_tokens=300,
    )
    result = response.choices[0].message.content
    try:
        # JSON puro
        parsed = json.loads(result)
    except Exception:
        # Se veio com markdown ou código, extrai só o JSON
        try:
            result_clean = result.split('```json')[1].split('```')[0].strip()
            parsed = json.loads(result_clean)
        except Exception as e:
            print("\n[ERRO] Falha ao interpretar JSON do GPT:", e)
            print("[PROMPT RECEBIDO]:", result)
            raise ValueError("Falha ao interpretar o JSON do prompt.")

    # Tira excesso (garante <77 tokens para CLIP/SD)
    final_prompt = " ".join(parsed["prompt"].split()[:75])
    return final_prompt


### Configurando processamento de imagem e definindo 'device' -> GPU = driver de vídeo | CPU = Processador local

In [ ]:
from diffusers import DiffusionPipeline
from PIL import Image
import torch
import os
from transformers import CLIPTokenizer

# Inicialize CLIPTokenizer uma vez no início
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch16")

def truncate_prompt_to_77_tokens(prompt: str) -> str:
    """
    Trunca o prompt para no máximo 77 tokens (limite do CLIP usado pelo Stable Diffusion).
    """
    tokens = tokenizer(prompt, truncation=True, max_length=77, return_tensors="pt")
    return tokenizer.decode(tokens['input_ids'][0], skip_special_tokens=True)

def validate_image(img_path, expected_size=(256, 256)):
    """
    Valida se a imagem existe, está no tamanho correto e não é preta/corrompida.
    """
    try:
        img = Image.open(img_path)
        if img.size != expected_size:
            print(f"[ERRO] Imagem gerada não é {expected_size}, mas {img.size}")
            return False
        if img.getbbox() is None:
            print("[ERRO] Imagem preta/vazia detectada!")
            return False
        extrema = img.getextrema()
        if all(e == (0, 0) for e in extrema):
            print("[ERRO] Imagem completamente preta!")
            return False
        return True
    except Exception as e:
        print(f"[ERRO] Imagem corrompida ou inválida: {e}")
        return False

def generate_avatar_image(
    image_prompt, 
    output_path="examples/source_image/generated_image.png", 
    size=(256, 256)
):
    """
    Gera avatar realista baseado em prompt usando Stable Diffusion, truncando o prompt para 77 tokens,
    redimensionando para 256x256 e validando a imagem para uso em SadTalker.

    Parâmetros:
        image_prompt (str): prompt detalhado em inglês
        output_path (str): caminho do arquivo para salvar a imagem
        size (tuple): tamanho exigido pelo SadTalker (default 256x256)

    Retorna:
        str: caminho absoluto da imagem gerada
    """
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"[INFO] Usando device: {device}")

    # Trunca prompt com precisão de tokens CLIP (NUNCA passará de 77)
    prompt_token_safe = truncate_prompt_to_77_tokens(image_prompt)
    print(f"[INFO] Prompt token-safe: {prompt_token_safe}")

    pipe = DiffusionPipeline.from_pretrained(
        "stabilityai/stable-diffusion-2-1",
        torch_dtype=torch.float32,
        safety_checker=None
    ).to(device)

    image = pipe(
        prompt_token_safe,
        num_inference_steps=30,
        guidance_scale=7.0
    ).images[0]

    # Converte sempre para RGB e redimensiona
    image = image.convert("RGB").resize(size, Image.LANCZOS)
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    image.save(output_path)
    print(f"[PIPELINE] Avatar image gerada em: {output_path}, tamanho: {image.size}")

    # Validação extra
    if not validate_image(output_path, size):
        raise RuntimeError("Imagem inválida ou preta/corrompida — ajuste o prompt e tente novamente!")

    return output_path


### Configurando processamento de áudio:

In [14]:
from gtts import gTTS
import os
import subprocess

def generate_voiceover(text, filename, lang='pt'):
    temp_mp3 = os.path.abspath("temp.mp3")
    filename = os.path.abspath(filename)
    os.makedirs(os.path.dirname(filename), exist_ok=True)

    tts = gTTS(text, lang=lang)
    tts.save(temp_mp3)

    try:
        result = subprocess.run(
            ["ffmpeg", "-y", "-i", temp_mp3, "-ar", "16000", "-ac", "1", filename],
            capture_output=True, text=True, encoding="utf-8", errors="ignore", check=True
        )
        print(f"[SUCCESS] Audio gerado: {filename}")
    except subprocess.CalledProcessError as e:
        print("[ERRO] ffmpeg falhou:")
        print(e.stdout)
        print(e.stderr)
        raise
    finally:
        if os.path.exists(temp_mp3):
            os.remove(temp_mp3)


### Configurando processamento de vídeo:

In [15]:
import os
import subprocess
from typing import List

def create_ai_influencer(
    image_path: str,
    audio_path: str,
    results_dir: str = "results",
    sad_talker_dir: str = "SadTalker"
) -> List[str]:
    """
    Executa a geração do vídeo do influenciador virtual, integrando imagem e áudio
    via SadTalker, com validação de diretórios, logging detalhado e tratamento de erros.

    Parâmetros:
        image_path (str): Caminho absoluto ou relativo da imagem do avatar (256x256 px, RGB).
        audio_path (str): Caminho absoluto ou relativo do arquivo de áudio (wav, 16kHz, mono).
        results_dir (str): Diretório de saída dos vídeos. Default = "results".
        sad_talker_dir (str): Diretório raiz do SadTalker (onde está o inference.py).

    Retorna:
        List[str]: Lista de arquivos gerados no diretório de resultados.

    Lança:
        RuntimeError: Se ocorrer falha na execução do SadTalker.
    """

    # 1. Garantir que diretórios existem e que caminhos são absolutos
    results_dir_abs = os.path.abspath(results_dir)
    sad_talker_dir_abs = os.path.abspath(sad_talker_dir)
    os.makedirs(results_dir_abs, exist_ok=True)

    # 2. Caminhos relativos para chamada do subprocess dentro do SadTalker
    image_rel = os.path.relpath(os.path.abspath(image_path), sad_talker_dir_abs)
    audio_rel = os.path.relpath(os.path.abspath(audio_path), sad_talker_dir_abs)
    results_rel = os.path.relpath(results_dir_abs, sad_talker_dir_abs)

    print(f"\n[PIPELINE] === EXECUTANDO SADTALKER ===")
    print(f"[PIPELINE] Imagem: {image_rel} | Áudio: {audio_rel} | Saída: {results_rel}")

    # 3. Execução robusta via subprocess, com logging detalhado de erro
    try:
        result = subprocess.run(
            [
                "python", "inference.py",
                "--driven_audio", audio_rel,
                "--source_image", image_rel,
                "--result_dir", results_rel,
                "--still",
                "--preprocess", "full",
                "--enhancer", "gfpgan"
            ],
            cwd=sad_talker_dir_abs,
            check=True,
            capture_output=True,
            text=True,
            encoding="utf-8"
        )
        print("[PIPELINE] SadTalker finalizado com sucesso.")
        print("[PIPELINE] STDOUT:", result.stdout)
    except subprocess.CalledProcessError as e:
        print("\n[ERRO PIPELINE] Falha ao executar SadTalker.")
        print("[ERRO] STDOUT:\n", e.stdout)
        print("[ERRO] STDERR:\n", e.stderr)
        raise RuntimeError("Erro ao executar SadTalker - verifique o log acima e revise parâmetros/paths/checkpoints.") from e

    # 4. Lista de arquivos gerados (pós-processamento)
    files = sorted(os.listdir(results_dir_abs))
    if not files:
        print("[WARNING] Nenhum arquivo gerado. Verifique o output/log de SadTalker.")
    else:
        print(f"[PIPELINE] Arquivos gerados em '{results_dir_abs}': {files}")
    return files

### Inicia processamento dos modelos de imagem, áudio e vídeo:

In [ ]:
def init(avatar_details, script, lang='pt'):
    """
    Pipeline corporativo para geração de vídeo de influenciador virtual:
    1. Gera um prompt em inglês, ultra-otimizado, sem exceder o limite de tokens do modelo de imagem.
    2. Gera a imagem do avatar (256x256, RGB) com alta fidelidade.
    3. Gera o áudio do voiceover no idioma desejado.
    4. Executa SadTalker para produzir o vídeo animado.

    Parâmetros:
        avatar_details (dict): características textuais do avatar (em português ou inglês).
        script (str): texto para o áudio do avatar.
        lang (str): idioma do áudio ('pt', 'en', etc).
    Retorna:
        res (list): arquivos gerados na pasta de resultados.
    """
    # 1. Gera prompt enxuto e perfeito para imagem realista de rosto
    characteristics = avatar_details.get('characteristics') or avatar_details.get('characterstics')
    prompt = get_prompt_for_image(characteristics)
    # Limita para garantir compatibilidade com CLIP/Stable Diffusion (máx 77 tokens)
    prompt_short = " ".join(prompt.split()[:75])
    print(f"[PIPELINE] Image prompt final: {prompt_short}")

    # 2. Gera a imagem com tamanho fixo (para o SadTalker)
    avatar_image_url = generate_avatar_image(prompt_short, size=(256, 256))
    print(f"[PIPELINE] Avatar image gerada em: {avatar_image_url}")

    # 3. Gera o áudio
    voice_path = os.path.join("examples", "driven_audio", "audio.wav")
    generate_voiceover(script, voice_path, lang=lang)
    print(f"[PIPELINE] Voiceover gerado em: {voice_path}")

    assert os.path.exists(avatar_image_url), f"Imagem não encontrada: {avatar_image_url}"
    assert os.path.exists(voice_path), f"Áudio não encontrado: {voice_path}"

    # 4. Executa SadTalker
    res = create_ai_influencer(avatar_image_url, voice_path)
    print(f"[PIPELINE] Arquivos gerados: {res}")
    return res

# Exemplo de uso – prompt ultra-otimizado para IA de imagem (em português, mas a função já converte para inglês no get_prompt_for_image):
avatar_info = {
    'characteristics': (
        "Mulher europeia jovem-adulta, postura elegante, liderança e empatia, olhos verdes intensos, "
        "óculos sofisticados com brilho sutil, cabelos castanho-escuros lisos e levemente ondulados nas pontas, "
        "maquiagem discreta e sofisticada realçando textura natural da pele clara, sorriso autêntico, "
        "jaqueta de couro preta fashion, fundo degradê cinza claro para escuro, iluminação de estúdio suave "
        "(efeito softbox frontal e lateral), estilo retrato corporativo editorial, rosto e ombros centralizados, "
        "resolução altíssima, atmosfera de inovação e confiança"
    )
}

init(
    avatar_info,
    "Olá Mundo. Eu sou a Inteligência do Google, muito prazer!",
    lang='pt'
)


[PIPELINE] Image prompt final: A young European woman with an elegant posture, exuding leadership and empathy. She has intense green eyes, sophisticated glasses with a subtle shine, and smooth dark brown hair with slight waves at the ends. Her makeup is discreet yet enhances her fair skin's natural texture. Wearing a fashionable black leather jacket, she smiles authentically against a soft gray gradient background, illuminated by gentle studio lighting, capturing an atmosphere of innovation and confidence.
[INFO] Usando device: cuda


Loading pipeline components...: 100%|██████████| 6/6 [00:02<00:00,  2.79it/s]
Token indices sequence length is longer than the specified maximum sequence length for this model (89 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['gentle studio lighting , capturing an atmosphere of innovation and confidence .']
100%|██████████| 30/30 [20:34<00:00, 41.16s/it]


[PIPELINE] Avatar image gerada em: examples/source_image/generated_image.png, tamanho: (256, 256)
[PIPELINE] Avatar image gerada em: examples/source_image/generated_image.png
[SUCCESS] Audio gerado: c:\Users\andre\OneDrive\Área de Trabalho\ESTUDOS\IA_GEN\AI-Influencer-Generator\examples\driven_audio\audio.wav
[PIPELINE] Voiceover gerado em: examples\driven_audio\audio.wav

[PIPELINE] === EXECUTANDO SADTALKER ===
[PIPELINE] Imagem: ..\examples\source_image\generated_image.png | Áudio: ..\examples\driven_audio\audio.wav | Saída: ..\results
[PIPELINE] SadTalker finalizado com sucesso.
[PIPELINE] STDOUT: [INFO] CUDA disponível: usando GPU.

[LOG] ======== SADTALKER PIPELINE START ========
[LOG] Imagem de entrada: ..\examples\source_image\generated_image.png
[LOG] Áudio de entrada: ..\examples\driven_audio\audio.wav
[LOG] Tamanho da imagem: (256, 256) (deve ser 256x256)
using safetensor as default
[LOG] 3DMM Extraction for source image
The generated video is named ..\results\2025_07_05_16.4

['2025_07_05_15.34.07', '2025_07_05_16.44.37.mp4']